In [48]:
import os
import django
import pandas as pd
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fxfutbol.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [49]:

from django.db import transaction
from analysis.models import PlayerPerformance
from analysis.models import MatchStats

class PlayersData:
    def __init__(self):
        pass
        
    def Data(self):
        
        fixture_data = MatchStats.objects.all().values()
        df = pd.DataFrame(list(fixture_data))
        stats_df = pd.json_normalize(df['stats'])
        mycols = ["lineups.match_id","lineups.response_code",
                  "match_info.data.event.homeTeam.id","match_info.data.event.awayTeam.id",
                  "match_info.data.event.homeTeam.name","match_info.data.event.awayTeam.name",
                  "lineups.data.home.players","lineups.data.away.players",
                  "lineups.data.home.missingPlayers","lineups.data.away.missingPlayers"]

        data = stats_df[mycols]
        data.columns = ["match_id","code_response",
                        "home_id","away_id",
                        "home_name","away_name",
                        "home_players","away_players",
                        "home_missing","away_missing"]
        
        home_players_expanded = data[['match_id', 'home_players','home_name','home_id']].explode('home_players').reset_index(drop=True)
        home_players_normalized = pd.json_normalize(home_players_expanded['home_players'])
        
        home_players_normalized['match_id'] = home_players_expanded['match_id']
        home_players_normalized['teamname'] = home_players_expanded['home_name']
        home_players_normalized['teamid'] = home_players_expanded['home_id']
        
        home_players_normalized['team'] = 'home'
        home_players_normalized.dropna(subset=['player.position'], inplace=True)
        away_players_expanded = data[['match_id', 'away_players','away_name','away_id']].explode('away_players').reset_index(drop=True)
        
        away_players_normalized = pd.json_normalize(away_players_expanded['away_players'])
        
        away_players_normalized['match_id'] = away_players_expanded['match_id']
        away_players_normalized['teamname'] = away_players_expanded['away_name']
        away_players_normalized['teamid'] = away_players_expanded['away_id']
        
        away_players_normalized['team'] = 'away'
        away_players_normalized.dropna(subset=['player.position'], inplace=True)

        players_normalized = pd.concat([home_players_normalized,away_players_normalized])

    
        original_columns = [
                            "shirtNumber", "jerseyNumber", "position", "substitute", "player.name", "player.slug", 
                            "player.shortName", "player.position", "player.jerseyNumber", "player.height", 
                            "player.userCount", "player.id", "player.country.alpha2", "player.country.alpha3", 
                            "player.country.name", "player.country.slug", "player.marketValueCurrency", 
                            "player.dateOfBirthTimestamp", "statistics.totalPass", "statistics.accuratePass", 
                            "statistics.totalLongBalls", "statistics.accurateLongBalls", "statistics.goalAssist", 
                            "statistics.duelWon", "statistics.wasFouled", "statistics.goodHighClaim", 
                            "statistics.savedShotsFromInsideTheBox", "statistics.saves", "statistics.minutesPlayed", 
                            "statistics.touches", "statistics.rating", "statistics.possessionLostCtrl", 
                            "statistics.ratingVersions.original", "statistics.ratingVersions.alternative", 
                            "statistics.totalCross", "statistics.accurateCross", "statistics.aerialWon", 
                            "statistics.duelLost", "statistics.challengeLost", "statistics.totalContest", 
                            "statistics.blockedScoringAttempt", "statistics.totalTackle", "player.firstName", 
                            "player.lastName", "statistics.aerialLost", "statistics.wonContest", 
                            "statistics.bigChanceCreated", "statistics.totalClearance", "statistics.interceptionWon", 
                            "statistics.fouls", "statistics.keyPass", "player.fieldTranslations.nameTranslation.ar", 
                            "player.fieldTranslations.shortNameTranslation.ar", "statistics.outfielderBlock", 
                            "statistics.dispossessed", "statistics.bigChanceMissed", "statistics.onTargetScoringAttempt", 
                            "captain", "statistics.shotOffTarget", "statistics.hitWoodwork", "statistics.totalOffside", 
                            "statistics.punches", "statistics.totalKeeperSweeper", "statistics.accurateKeeperSweeper", 
                            "statistics.goals", "statistics.errorLeadToAShot", "statistics.penaltyConceded", 
                            "statistics.lastManTackle", "statistics.ownGoals", "statistics.expectedGoals", 
                            "statistics.expectedAssists", "statistics.penaltyWon", "statistics.penaltyMiss", 
                            "statistics.clearanceOffLine", "statistics.goalsPrevented", "statistics.errorLeadToAGoal", 
                            "statistics.penaltySave", "statistics.penaltyShootoutSave", "statistics.penaltyShootoutGoal", 
                            "statistics.penaltyShootoutMiss", "match_id", "teamname", "teamid", "team"
                        ]

        
        for col in original_columns:
               if col not in players_normalized.columns:
                    players_normalized[col] = 0
            
        players = players_normalized[original_columns]


        new_columns = [
                        "shirt_number", "jersey_number", "position", "substitute", "player_name", "player_slug", 
                        "player_short_name", "player_position", "player_jersey_number", "player_height", 
                        "player_user_count", "player_id", "country_alpha2", "country_alpha3", 
                        "country_name", "country_slug", "market_value_currency", 
                        "date_of_birth_timestamp", "PAS_total_passes", "PAS_accurate_passes", 
                        "PAS_long_balls", "PAS_accurate_long_balls", "ATT_goal_assist", 
                        "MUC_duel_won", "MUC_fouls_suffered", "KAL_good_high_claim", 
                        "KAL_saved_shots_from_inside_the_box", "KAL_saves", "minutes_played", 
                        "touches", "rating", "MUC_possession_lost", 
                        "rating_original", "rating_alternative", 
                        "PAS_crosses", "PAS_accurate_crosses", "MUC_aerial_duel_won", 
                        "MUC_duel_lost", "MUC_challenge_lost", "MUC_total_contest", 
                        "DEF_blocked_scoring_attempt", "DEF_tackles", "player_first_name", 
                        "player_last_name", "MUC_aerial_lost", "MUC_won_contest", 
                        "ATT_big_chance_created", "DEF_clearances", "DEF_interceptions", 
                        "MUC_fouls_committed", "PAS_key_passes", "name_translation_ar", 
                        "short_name_translation_ar", "DEF_outfielder_block", 
                        "MUC_dispossessed", "ATT_big_chance_missed", "ATT_on_target_scoring_attempt", 
                        "captain", "ATT_shot_off_target", "ATT_hit_woodwork", "MUC_offsides", 
                        "KAL_punches", "KAL_total_keeper_sweeper", "KAL_accurate_keeper_sweeper", 
                        "ATT_goals", "MUC_error_lead_to_a_shot", "DEF_penalty_conceded", 
                        "DEF_last_man_tackle", "DEF_own_goals", "ATT_expected_goals", 
                        "ATT_expected_assists", "ATT_penalty_won", "ATT_penalty_miss", 
                        "DEF_clearance_off_line", "KAL_goals_prevented", "MUC_error_lead_to_a_goal", 
                        "KAL_penalty_save", "KAL_penalty_shootout_save", "KAL_penalty_shootout_goal", 
                        "KAL_penalty_shootout_miss", "match_id", "team_name", "team_id", "team"
                    ]


        players.columns = new_columns
        playersdata = players.fillna(0)
        playersdata.replace('', 0, inplace=True)

        
         
        return playersdata

    def add_to_database(self):

        players = self.Data()

    # Veri eklemek veya güncellemek için döngüyle her satırı işliyoruz
        for _, row in players.iterrows():
            defaults = {
                
                'shirt_number': row.get('shirt_number', None) if pd.notna(row.get('shirt_number')) else None,
                'jersey_number': row.get('jersey_number', None) if pd.notna(row.get('jersey_number')) else None,
                'position': row.get('position', None) if pd.notna(row.get('position')) else None,
                'substitute': row.get('substitute', None) if pd.notna(row.get('substitute')) else None,
                'player_name': row.get('player_name', None) if pd.notna(row.get('player_name')) else None,
                'player_slug': row.get('player_slug', None) if pd.notna(row.get('player_slug')) else None,
                'player_short_name': row.get('player_short_name', None) if pd.notna(row.get('player_short_name')) else None,
                'player_position': row.get('player_position', None) if pd.notna(row.get('player_position')) else None,
                'player_jersey_number': row.get('player_jersey_number', None) if pd.notna(row.get('player_jersey_number')) else None,
                'player_height': row.get('player_height', None) if pd.notna(row.get('player_height')) else None,
                'player_user_count': row.get('player_user_count', None) if pd.notna(row.get('player_user_count')) else None,
                'country_alpha2': row.get('country_alpha2', None) if pd.notna(row.get('country_alpha2')) else None,
                'country_alpha3': row.get('country_alpha3', None) if pd.notna(row.get('country_alpha3')) else None,
                'country_name': row.get('country_name', None) if pd.notna(row.get('country_name')) else None,
                'country_slug': row.get('country_slug', None) if pd.notna(row.get('country_slug')) else None,
                'market_value_currency': row.get('market_value_currency', None) if pd.notna(row.get('market_value_currency')) else None,
                'date_of_birth_timestamp': row.get('date_of_birth_timestamp', None) if pd.notna(row.get('date_of_birth_timestamp')) else None,
            
                # ATT (Attack) Stats
                'ATT_goals': row.get('ATT_goals', None) if pd.notna(row.get('ATT_goals')) else None,
                'ATT_goal_assist': row.get('ATT_goal_assist', None) if pd.notna(row.get('ATT_goal_assist')) else None,
                'ATT_shot_off_target': row.get('ATT_shot_off_target', None) if pd.notna(row.get('ATT_shot_off_target')) else None,
                'ATT_hit_woodwork': row.get('ATT_hit_woodwork', None) if pd.notna(row.get('ATT_hit_woodwork')) else None,
                'ATT_big_chance_created': row.get('ATT_big_chance_created', None) if pd.notna(row.get('ATT_big_chance_created')) else None,
                'ATT_big_chance_missed': row.get('ATT_big_chance_missed', None) if pd.notna(row.get('ATT_big_chance_missed')) else None,
                'ATT_on_target_scoring_attempt': row.get('ATT_on_target_scoring_attempt', None) if pd.notna(row.get('ATT_on_target_scoring_attempt')) else None,
                'ATT_expected_goals': row.get('ATT_expected_goals', None) if pd.notna(row.get('ATT_expected_goals')) else None,
                'ATT_expected_assists': row.get('ATT_expected_assists', None) if pd.notna(row.get('ATT_expected_assists')) else None,
                'ATT_penalty_won': row.get('ATT_penalty_won', None) if pd.notna(row.get('ATT_penalty_won')) else None,
                'ATT_penalty_miss': row.get('ATT_penalty_miss', None) if pd.notna(row.get('ATT_penalty_miss')) else None,
            
                # PAS (Pass) Stats
                'PAS_total_passes': row.get('PAS_total_passes', None) if pd.notna(row.get('PAS_total_passes')) else None,
                'PAS_accurate_passes': row.get('PAS_accurate_passes', None) if pd.notna(row.get('PAS_accurate_passes')) else None,
                'PAS_key_passes': row.get('PAS_key_passes', None) if pd.notna(row.get('PAS_key_passes')) else None,
                'PAS_crosses': row.get('PAS_crosses', None) if pd.notna(row.get('PAS_crosses')) else None,
                'PAS_accurate_crosses': row.get('PAS_accurate_crosses', None) if pd.notna(row.get('PAS_accurate_crosses')) else None,
                'PAS_long_balls': row.get('PAS_long_balls', None) if pd.notna(row.get('PAS_long_balls')) else None,
                'PAS_accurate_long_balls': row.get('PAS_accurate_long_balls', None) if pd.notna(row.get('PAS_accurate_long_balls')) else None,
            
                # MUC (Duels and Fouls) Stats
                'MUC_duel_won': row.get('MUC_duel_won', None) if pd.notna(row.get('MUC_duel_won')) else None,
                'MUC_aerial_duel_won': row.get('MUC_aerial_duel_won', None) if pd.notna(row.get('MUC_aerial_duel_won')) else None,
                'MUC_duel_lost': row.get('MUC_duel_lost', None) if pd.notna(row.get('MUC_duel_lost')) else None,
                'MUC_aerial_lost': row.get('MUC_aerial_lost', None) if pd.notna(row.get('MUC_aerial_lost')) else None,
                'MUC_won_contest': row.get('MUC_won_contest', None) if pd.notna(row.get('MUC_won_contest')) else None,
                'MUC_challenge_lost': row.get('MUC_challenge_lost', None) if pd.notna(row.get('MUC_challenge_lost')) else None,
                'MUC_total_contest': row.get('MUC_total_contest', None) if pd.notna(row.get('MUC_total_contest')) else None,
                'MUC_possession_lost': row.get('MUC_possession_lost', None) if pd.notna(row.get('MUC_possession_lost')) else None,
                'MUC_fouls_committed': row.get('MUC_fouls_committed', None) if pd.notna(row.get('MUC_fouls_committed')) else None,
                'MUC_fouls_suffered': row.get('MUC_fouls_suffered', None) if pd.notna(row.get('MUC_fouls_suffered')) else None,
                'MUC_offsides': row.get('MUC_offsides', None) if pd.notna(row.get('MUC_offsides')) else None,
                'MUC_dispossessed': row.get('MUC_dispossessed', None) if pd.notna(row.get('MUC_dispossessed')) else None,
                'MUC_error_lead_to_a_shot': row.get('MUC_error_lead_to_a_shot', None) if pd.notna(row.get('MUC_error_lead_to_a_shot')) else None,
                'MUC_error_lead_to_a_goal': row.get('MUC_error_lead_to_a_goal', None) if pd.notna(row.get('MUC_error_lead_to_a_goal')) else None,

    # DEF (Defensive) Stats
                'DEF_clearances': row.get('DEF_clearances', None) if pd.notna(row.get('DEF_clearances')) else None,
                'DEF_blocked_scoring_attempt': row.get('DEF_blocked_scoring_attempt', None) if pd.notna(row.get('DEF_blocked_scoring_attempt')) else None,
                'DEF_tackles': row.get('DEF_tackles', None) if pd.notna(row.get('DEF_tackles')) else None,
                'DEF_interceptions': row.get('DEF_interceptions', None) if pd.notna(row.get('DEF_interceptions')) else None,
                'DEF_last_man_tackle': row.get('DEF_last_man_tackle', None) if pd.notna(row.get('DEF_last_man_tackle')) else None,
                'DEF_penalty_conceded': row.get('DEF_penalty_conceded', None) if pd.notna(row.get('DEF_penalty_conceded')) else None,
                'DEF_own_goals': row.get('DEF_own_goals', None) if pd.notna(row.get('DEF_own_goals')) else None,
                'DEF_clearance_off_line': row.get('DEF_clearance_off_line', None) if pd.notna(row.get('DEF_clearance_off_line')) else None,
                          'DEF_outfielder_block': row.get('DEF_outfielder_block', None) if pd.notna(row.get('DEF_outfielder_block')) else None,
            
                # KAL (Kaleci) Stats
                'KAL_good_high_claim': row.get('KAL_good_high_claim', None) if pd.notna(row.get('KAL_good_high_claim')) else None,
                'KAL_saved_shots_from_inside_the_box': row.get('KAL_saved_shots_from_inside_the_box', None) if pd.notna(row.get('KAL_saved_shots_from_inside_the_box')) else None,
                'KAL_saves': row.get('KAL_saves', None) if pd.notna(row.get('KAL_saves')) else None,
                'KAL_punches': row.get('KAL_punches', None) if pd.notna(row.get('KAL_punches')) else None,
                'KAL_total_keeper_sweeper': row.get('KAL_total_keeper_sweeper', None) if pd.notna(row.get('KAL_total_keeper_sweeper')) else None,
                'KAL_accurate_keeper_sweeper': row.get('KAL_accurate_keeper_sweeper', None) if pd.notna(row.get('KAL_accurate_keeper_sweeper')) else None,
                'KAL_goals_prevented': row.get('KAL_goals_prevented', None) if pd.notna(row.get('KAL_goals_prevented')) else None,
                'KAL_penalty_save': row.get('KAL_penalty_save', None) if pd.notna(row.get('KAL_penalty_save')) else None,
                'KAL_penalty_shootout_save': row.get('KAL_penalty_shootout_save', None) if pd.notna(row.get('KAL_penalty_shootout_save')) else None,
                'KAL_penalty_shootout_goal': row.get('KAL_penalty_shootout_goal', None) if pd.notna(row.get('KAL_penalty_shootout_goal')) else None,
                'KAL_penalty_shootout_miss': row.get('KAL_penalty_shootout_miss', None) if pd.notna(row.get('KAL_penalty_shootout_miss')) else None,

                # Match and Team Information
                'team_name': row.get('team_name', None) if pd.notna(row.get('team_name')) else None,
                'team_id': row.get('team_id', None) if pd.notna(row.get('team_id')) else None,
                'team': row.get('team', None) if pd.notna(row.get('team')) else None,
            
                # Other Statistics
                'minutes_played': row.get('minutes_played', None) if pd.notna(row.get('minutes_played')) else None,
                'touches': row.get('touches', None) if pd.notna(row.get('touches')) else None,
                'rating': row.get('rating', None) if pd.notna(row.get('rating')) else None,
                'rating_original': row.get('rating_original', None) if pd.notna(row.get('rating_original')) else None,
                'rating_alternative': row.get('rating_alternative', None) if pd.notna(row.get('rating_alternative')) else None,
            
                # Additional Player Information
                'player_first_name': row.get('player_first_name', None) if pd.notna(row.get('player_first_name')) else None,
                'player_last_name': row.get('player_last_name', None) if pd.notna(row.get('player_last_name')) else None,

                            }

            PlayerPerformance.objects.update_or_create(
                    match_id=row.get('match_id'),
                    player_id=row.get('player_id'),
                    defaults=defaults
                          )
        print("Player Performance Data Guncellendi.")

        


d = PlayersData()
data = d.Data()
data

,shirt_number,jersey_number,position,substitute,player_name,player_slug,player_short_name,player_position,player_jersey_number,player_height,...,KAL_goals_prevented,MUC_error_lead_to_a_goal,KAL_penalty_save,KAL_penalty_shootout_save,KAL_penalty_shootout_goal,KAL_penalty_shootout_miss,match_id,team_name,team_id,team
0,24,24,G,False,André Onana,andre-onana,A. Onana,G,24,190.0,...,0.9171,0.0,0.0,0,0,0,12436870,Manchester United,35,home
1,3,3,D,False,Noussair Mazraoui,noussair-mazraoui,N. Mazraoui,D,3,183.0,...,0.0000,0.0,0.0,0,0,0,12436870,Manchester United,35,home
2,5,5,D,False,Harry Maguire,harry-maguire,H. Maguire,D,5,194.0,...,0.0000,0.0,0.0,0,0,0,12436870,Manchester United,35,home
3,6,6,D,False,Lisandro Martínez,lisandro-martinez,L. Martínez,D,6,175.0,...,0.0000,0.0,0.0,0,0,0,12436870,Manchester United,35,home
4,20,20,D,False,Diogo Dalot,diogo-dalot,D. Dalot,D,20,184.0,...,0.0000,0.0,0.0,0,0,0,12436870,Manchester United,35,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5049,4,4,D,True,Juan Berrocal,juan-berrocal,J. Berrocal,D,4,187.0,...,0.0000,0.0,0.0,0,0,0,12778688,Getafe,2859,away
5050,33,33,D,True,Guillem Trilla,guillem-trilla,G. Trilla,D,33,0.0,...,0.0000,0.0,0.0,0,0,0,12778688,Getafe,2859,away
5051,22,22,D,True,Domingos Duarte,domingos-duarte,D. Duarte,D,22,191.0,...,0.0000,0.0,0.0,0,0,0,12778688,Getafe,2859,away
5052,31,31,M,True,John Finn,john-finn,J. Finn,M,31,193.0,...,0.0000,0.0,0.0,0,0,0,12778688,Getafe,2859,away


In [50]:
d = PlayersData()
data = d.Data()

float_columns = data.select_dtypes(include=['float']).columns.tolist()

h = data[data["team"]=="home"].groupby(["match_id","team_name","team_id"])[float_columns ].sum().reset_index()
a = data[data["team"]=="away"].groupby(["match_id","team_name","team_id"])[float_columns ].sum().reset_index()
f = pd.merge(h,a,on="match_id")
f

,match_id,team_name_x,team_id_x,player_height_x,date_of_birth_timestamp_x,PAS_total_passes_x,PAS_accurate_passes_x,PAS_long_balls_x,PAS_accurate_long_balls_x,ATT_goal_assist_x,...,DEF_last_man_tackle_y,DEF_own_goals_y,ATT_expected_goals_y,ATT_expected_assists_y,ATT_penalty_won_y,ATT_penalty_miss_y,DEF_clearance_off_line_y,KAL_goals_prevented_y,MUC_error_lead_to_a_goal_y,KAL_penalty_save_y
0,12436870,Manchester United,35,3685.0,1.770483e+10,482.0,408.0,40.0,23.0,1.0,...,0.0,0.0,0.4353,0.442857,0.0,0.0,0.0,1.0671,0.0,0.0
1,12436871,Ipswich Town,32,3631.0,1.756875e+10,347.0,272.0,47.0,19.0,0.0,...,1.0,0.0,2.6589,2.438838,0.0,0.0,0.0,0.2438,0.0,0.0
2,12436872,Arsenal,42,3640.0,1.819403e+10,420.0,357.0,37.0,17.0,2.0,...,0.0,0.0,0.4694,0.255447,0.0,0.0,0.0,-0.2967,0.0,0.0
3,12436873,Everton,48,3671.0,1.735439e+10,344.0,266.0,82.0,45.0,0.0,...,0.0,0.0,1.4304,0.954460,0.0,0.0,0.0,0.0498,0.0,0.0
4,12436874,Newcastle United,39,3694.0,1.693250e+10,181.0,117.0,39.0,12.0,1.0,...,0.0,0.0,1.8218,1.605212,0.0,0.0,0.0,-0.3686,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,12499321,Lazio,2699,4006.0,1.922789e+10,419.0,360.0,38.0,20.0,2.0,...,0.0,0.0,0.5489,0.179900,0.0,0.0,0.0,-0.0238,0.0,1.0
226,12499322,Monza,2729,3661.0,1.824405e+10,310.0,234.0,55.0,31.0,0.0,...,0.0,0.0,1.0720,0.768870,0.0,0.0,0.0,0.1869,0.0,0.0
227,12499323,Napoli,2714,4207.0,2.078568e+10,424.0,364.0,36.0,22.0,2.0,...,0.0,0.0,0.3688,0.544384,0.0,0.0,0.0,-0.3845,0.0,0.0
228,12499324,Udinese,2695,4104.0,2.011954e+10,439.0,351.0,51.0,18.0,0.0,...,0.0,0.0,0.5066,0.244424,0.0,0.0,0.0,-0.0990,0.0,0.0


In [51]:
import pandas as pd

from mymodel.models import TeamPerformance  # TeamPerformance modelini import edin

class PerformanceTeam:

    def __init__(self):
        pass

    def process(self):

        d = PlayersData()
        data = d.Data()

        float_columns = data.select_dtypes(include=['float']).columns.tolist()

        # Home ve Away takımlarını gruplandırarak sum() ile toplama işlemi
        home_data = data[data["team"] == "home"].groupby(
            ["match_id", "team_name", "team_id"]
        )[float_columns].sum().reset_index()

        away_data = data[data["team"] == "away"].groupby(
            ["match_id", "team_name", "team_id"]
        )[float_columns].sum().reset_index()

        # Home ve Away verilerini match_id'ye göre birleştir
        merged_data = pd.merge(home_data, away_data, on="match_id", suffixes=('_x', '_y'))

        return merged_data

    def save_to_model(self):
        
        merged_datax=self.process()
        
        TeamPerformance.objects.all().delete()

        # İterate ederek her satırı modelimize ekliyoruz
        for index, row in merged_datax.iterrows():
            team_performance = TeamPerformance(
                match_id=row['match_id'],
                team_name_x=row['team_name_x'],
                team_id_x=row['team_id_x'],
                player_height_x=row.get('player_height_x', 0),
                PAS_total_passes_x=row.get('PAS_total_passes_x', 0),
                PAS_accurate_passes_x=row.get('PAS_accurate_passes_x', 0),
                PAS_long_balls_x=row.get('PAS_long_balls_x', 0),
                PAS_accurate_long_balls_x=row.get('PAS_accurate_long_balls_x', 0),
                ATT_goal_assist_x=row.get('ATT_goal_assist_x', 0),
                MUC_duel_won_x=row.get('MUC_duel_won_x', 0),
                MUC_fouls_suffered_x=row.get('MUC_fouls_suffered_x', 0),
                KAL_good_high_claim_x=row.get('KAL_good_high_claim_x', 0),
                KAL_saved_shots_from_inside_the_box_x=row.get('KAL_saved_shots_from_inside_the_box_x', 0),
                KAL_saves_x=row.get('KAL_saves_x', 0),
                minutes_played_x=row.get('minutes_played_x', 0),
                touches_x=row.get('touches_x', 0),
                rating_x=row.get('rating_x', 0),
                MUC_possession_lost_x=row.get('MUC_possession_lost_x', 0),
                rating_original_x=row.get('rating_original_x', 0),
                rating_alternative_x=row.get('rating_alternative_x', 0),
                PAS_crosses_x=row.get('PAS_crosses_x', 0),
                PAS_accurate_crosses_x=row.get('PAS_accurate_crosses_x', 0),
                MUC_aerial_duel_won_x=row.get('MUC_aerial_duel_won_x', 0),
                MUC_duel_lost_x=row.get('MUC_duel_lost_x', 0),
                MUC_challenge_lost_x=row.get('MUC_challenge_lost_x', 0),
                MUC_total_contest_x=row.get('MUC_total_contest_x', 0),
                DEF_blocked_scoring_attempt_x=row.get('DEF_blocked_scoring_attempt_x', 0),
                DEF_tackles_x=row.get('DEF_tackles_x', 0),
                MUC_aerial_lost_x=row.get('MUC_aerial_lost_x', 0),
                MUC_won_contest_x=row.get('MUC_won_contest_x', 0),
                ATT_big_chance_created_x=row.get('ATT_big_chance_created_x', 0),
                DEF_clearances_x=row.get('DEF_clearances_x', 0),
                DEF_interceptions_x=row.get('DEF_interceptions_x', 0),
                MUC_fouls_committed_x=row.get('MUC_fouls_committed_x', 0),
                PAS_key_passes_x=row.get('PAS_key_passes_x', 0),
                DEF_outfielder_block_x=row.get('DEF_outfielder_block_x', 0),
                MUC_dispossessed_x=row.get('MUC_dispossessed_x', 0),
                ATT_big_chance_missed_x=row.get('ATT_big_chance_missed_x', 0),
                ATT_on_target_scoring_attempt_x=row.get('ATT_on_target_scoring_attempt_x', 0),
                ATT_shot_off_target_x=row.get('ATT_shot_off_target_x', 0),
                ATT_hit_woodwork_x=row.get('ATT_hit_woodwork_x', 0),
                MUC_offsides_x=row.get('MUC_offsides_x', 0),
                KAL_punches_x=row.get('KAL_punches_x', 0),
                KAL_total_keeper_sweeper_x=row.get('KAL_total_keeper_sweeper_x', 0),
                KAL_accurate_keeper_sweeper_x=row.get('KAL_accurate_keeper_sweeper_x', 0),
                ATT_goals_x=row.get('ATT_goals_x', 0),
                MUC_error_lead_to_a_shot_x=row.get('MUC_error_lead_to_a_shot_x', 0),
                DEF_penalty_conceded_x=row.get('DEF_penalty_conceded_x', 0),
                DEF_last_man_tackle_x=row.get('DEF_last_man_tackle_x', 0),
                DEF_own_goals_x=row.get('DEF_own_goals_x', 0),
                ATT_expected_goals_x=row.get('ATT_expected_goals_x', 0),
                ATT_expected_assists_x=row.get('ATT_expected_assists_x', 0),
                ATT_penalty_won_x=row.get('ATT_penalty_won_x', 0),
                ATT_penalty_miss_x=row.get('ATT_penalty_miss_x', 0),
                DEF_clearance_off_line_x=row.get('DEF_clearance_off_line_x', 0),
                KAL_goals_prevented_x=row.get('KAL_goals_prevented_x', 0),
                MUC_error_lead_to_a_goal_x=row.get('MUC_error_lead_to_a_goal_x', 0),
                KAL_penalty_save_x=row.get('KAL_penalty_save_x', 0),
                # Away Team Fields (team_y)
                team_name_y=row['team_name_y'],
                team_id_y=row['team_id_y'],
                player_height_y=row.get('player_height_y', 0),
                PAS_total_passes_y=row.get('PAS_total_passes_y', 0),
                PAS_accurate_passes_y=row.get('PAS_accurate_passes_y', 0),
                PAS_long_balls_y=row.get('PAS_long_balls_y', 0),
                PAS_accurate_long_balls_y=row.get('PAS_accurate_long_balls_y', 0),
                ATT_goal_assist_y=row.get('ATT_goal_assist_y', 0),
                MUC_duel_won_y=row.get('MUC_duel_won_y', 0),
                MUC_fouls_suffered_y=row.get('MUC_fouls_suffered_y', 0),
                KAL_good_high_claim_y=row.get('KAL_good_high_claim_y', 0),
                KAL_saved_shots_from_inside_the_box_y=row.get('KAL_saved_shots_from_inside_the_box_y', 0),
                KAL_saves_y=row.get('KAL_saves_y', 0),
                minutes_played_y=row.get('minutes_played_y', 0),
                touches_y=row.get('touches_y', 0),
                rating_y=row.get('rating_y', 0),
                MUC_possession_lost_y=row.get('MUC_possession_lost_y', 0),
                rating_original_y=row.get('rating_original_y', 0),
                rating_alternative_y=row.get('rating_alternative_y', 0),
                PAS_crosses_y=row.get('PAS_crosses_y', 0),
                PAS_accurate_crosses_y=row.get('PAS_accurate_crosses_y', 0),
                MUC_aerial_duel_won_y=row.get('MUC_aerial_duel_won_y', 0),
                MUC_duel_lost_y=row.get('MUC_duel_lost_y', 0),
                MUC_challenge_lost_y=row.get('MUC_challenge_lost_y', 0),
                MUC_total_contest_y=row.get('MUC_total_contest_y', 0),
                DEF_blocked_scoring_attempt_y=row.get('DEF_blocked_scoring_attempt_y', 0),
                DEF_tackles_y=row.get('DEF_tackles_y', 0),
                
                MUC_aerial_lost_y=row.get('MUC_aerial_lost_y', 0),
                MUC_won_contest_y=row.get('MUC_won_contest_y', 0),
                ATT_big_chance_created_y=row.get('ATT_big_chance_created_y', 0),
                DEF_clearances_y=row.get('DEF_clearances_y', 0),
                DEF_interceptions_y=row.get('DEF_interceptions_y', 0),
                MUC_fouls_committed_y=row.get('MUC_fouls_committed_y', 0),
                PAS_key_passes_y=row.get('PAS_key_passes_y', 0),
                DEF_outfielder_block_y=row.get('DEF_outfielder_block_y', 0),
                MUC_dispossessed_y=row.get('MUC_dispossessed_y', 0),
                ATT_big_chance_missed_y=row.get('ATT_big_chance_missed_y', 0),
                ATT_on_target_scoring_attempt_y=row.get('ATT_on_target_scoring_attempt_y', 0),
                ATT_shot_off_target_y=row.get('ATT_shot_off_target_y', 0),
                ATT_hit_woodwork_y=row.get('ATT_hit_woodwork_y', 0),
                MUC_offsides_y=row.get('MUC_offsides_y', 0),
                KAL_punches_y=row.get('KAL_punches_y', 0),
                KAL_total_keeper_sweeper_y=row.get('KAL_total_keeper_sweeper_y', 0),
                KAL_accurate_keeper_sweeper_y=row.get('KAL_accurate_keeper_sweeper_y', 0),
                ATT_goals_y=row.get('ATT_goals_y', 0),
                MUC_error_lead_to_a_shot_y=row.get('MUC_error_lead_to_a_shot_y', 0),
                DEF_penalty_conceded_y=row.get('DEF_penalty_conceded_y', 0),
                DEF_last_man_tackle_y=row.get('DEF_last_man_tackle_y', 0),
                DEF_own_goals_y=row.get('DEF_own_goals_y', 0),
                ATT_expected_goals_y=row.get('ATT_expected_goals_y', 0),
                ATT_expected_assists_y=row.get('ATT_expected_assists_y', 0),
                ATT_penalty_won_y=row.get('ATT_penalty_won_y', 0),
                ATT_penalty_miss_y=row.get('ATT_penalty_miss_y', 0),
                DEF_clearance_off_line_y=row.get('DEF_clearance_off_line_y', 0),
                KAL_goals_prevented_y=row.get('KAL_goals_prevented_y', 0),
                MUC_error_lead_to_a_goal_y=row.get('MUC_error_lead_to_a_goal_y', 0),
                KAL_penalty_save_y=row.get('KAL_penalty_save_y', 0)

                  )
            # Kaydet
            team_performance.save()
        print("veriler eklendi")
                
performance = PerformanceTeam()
performance.save_to_model()

veriler eklendi


In [77]:
fixture_data = TeamPerformance.objects.all().values()
df = pd.DataFrame(list(fixture_data))
df.fillna(0, inplace=True)
df

,id,match_id,team_name_x,team_id_x,player_height_x,PAS_total_passes_x,PAS_accurate_passes_x,PAS_long_balls_x,PAS_accurate_long_balls_x,ATT_goal_assist_x,...,DEF_last_man_tackle_y,DEF_own_goals_y,ATT_expected_goals_y,ATT_expected_assists_y,ATT_penalty_won_y,ATT_penalty_miss_y,DEF_clearance_off_line_y,KAL_goals_prevented_y,MUC_error_lead_to_a_goal_y,KAL_penalty_save_y
0,481,12436870,Manchester United,35,3685.0,482,408,40,23,1,...,0,0,0.4353,0.442857,0,0,0,1.0671,0,0
1,482,12436871,Ipswich Town,32,3631.0,347,272,47,19,0,...,1,0,2.6589,2.438838,0,0,0,0.2438,0,0
2,483,12436872,Arsenal,42,3640.0,420,357,37,17,2,...,0,0,0.4694,0.255447,0,0,0,-0.2967,0,0
3,484,12436873,Everton,48,3671.0,344,266,82,45,0,...,0,0,1.4304,0.954460,0,0,0,0.0498,0,0
4,485,12436874,Newcastle United,39,3694.0,181,117,39,12,1,...,0,0,1.8218,1.605212,0,0,0,-0.3686,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,706,12499321,Lazio,2699,4006.0,419,360,38,20,2,...,0,0,0.5489,0.179900,0,0,0,-0.0238,0,1
226,707,12499322,Monza,2729,3661.0,310,234,55,31,0,...,0,0,1.0720,0.768870,0,0,0,0.1869,0,0
227,708,12499323,Napoli,2714,4207.0,424,364,36,22,2,...,0,0,0.3688,0.544384,0,0,0,-0.3845,0,0
228,709,12499324,Udinese,2695,4104.0,439,351,51,18,0,...,0,0,0.5066,0.244424,0,0,0,-0.0990,0,0


In [98]:
X = df.drop(["id","match_id",
         'team_name_x',
     'team_id_x', 
    'team_name_y',
     'team_id_y',
    'player_height_y',
    'rating_original_y',
     'rating_alternative_y',
    'team_name_x',
     'team_id_x',
    'player_height_x',
    'rating_original_x',
     'rating_alternative_x',
         'minutes_played_x',
         'minutes_played_y',
         'ATT_goals_x',
         'ATT_goals_y',
        
    ],axis=1)


X



,PAS_total_passes_x,PAS_accurate_passes_x,PAS_long_balls_x,PAS_accurate_long_balls_x,ATT_goal_assist_x,MUC_duel_won_x,MUC_fouls_suffered_x,KAL_good_high_claim_x,KAL_saved_shots_from_inside_the_box_x,KAL_saves_x,...,DEF_last_man_tackle_y,DEF_own_goals_y,ATT_expected_goals_y,ATT_expected_assists_y,ATT_penalty_won_y,ATT_penalty_miss_y,DEF_clearance_off_line_y,KAL_goals_prevented_y,MUC_error_lead_to_a_goal_y,KAL_penalty_save_y
0,482,408,40,23,1,58,10,2,1,2,...,0,0,0.4353,0.442857,0,0,0,1.0671,0,0
1,347,272,47,19,0,51,17,1,3,3,...,1,0,2.6589,2.438838,0,0,0,0.2438,0,0
2,420,357,37,17,2,58,14,1,3,3,...,0,0,0.4694,0.255447,0,0,0,-0.2967,0,0
3,344,266,82,45,0,54,8,1,2,2,...,0,0,1.4304,0.954460,0,0,0,0.0498,0,0
4,181,117,39,12,1,40,16,0,3,3,...,0,0,1.8218,1.605212,0,0,0,-0.3686,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,419,360,38,20,2,64,16,0,1,1,...,0,0,0.5489,0.179900,0,0,0,-0.0238,0,1
226,310,234,55,31,0,55,13,0,3,4,...,0,0,1.0720,0.768870,0,0,0,0.1869,0,0
227,424,364,36,22,2,37,9,1,0,1,...,0,0,0.3688,0.544384,0,0,0,-0.3845,0,0
228,439,351,51,18,0,54,15,2,2,3,...,0,0,0.5066,0.244424,0,0,0,-0.0990,0,0


In [99]:
y1 = df['ATT_goals_x']
y2 = df['ATT_goals_y']

y1

0      1
1      0
2      2
3      0
4      1
      ..
225    2
226    1
227    3
228    1
229    2
Name: ATT_goals_x, Length: 230, dtype: int64